# GPS
GPS Benchmark

In [2]:
import sys
import os
import random

# Add the absolute path to the src directory 
os.chdir('../src')
src_path = os.path.abspath('../src/')
sys.path.insert(0, src_path)

from setup import *

In [3]:
# setting global variables
EPOCHS = 80

# IMDB Data

In [ ]:
# load data
imdb_train_loader, imdb_test_loader = load_imdb()
imdb_dataset = imdb_train_loader.dataset
num_node_features = imdb_dataset[0].num_node_features

# input params
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
attn_kwargs = {'dropout': 0.5}
model = GPSGraph(channels=64,\
             num_node_features = num_node_features, \
                num_layers=4, attn_type='performer', \
                    attn_kwargs=attn_kwargs).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, min_lr=0.00001)

# train test acc
for epoch in range(EPOCHS):
    loss = train_gps_graph(model, imdb_train_loader, optimizer, device)
    test_acc = test_gps_graph(model, imdb_test_loader, device)
    train_acc = test_gps_graph(model, imdb_train_loader, device)
    scheduler.step(loss)

print(f'Final Loss: {loss:.4f}, Final Train Acc: {train_acc:.4f}, Final Test Acc: {test_acc:.4f}')


Processing...
Done!


Final Loss: 0.4603, Final Train Acc: 0.7875, Final Test Acc: 0.7450


# ENZYME data

In [ ]:
# load data
enzyme_train_loader, enzyme_test_loader = load_enzyme()
enzyme_dataset = enzyme_train_loader.dataset
num_node_features = enzyme_dataset[0].num_node_features

# input params
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
attn_kwargs = {'dropout': 0.5}
model = GPSGraph(channels=132,\
             num_node_features = num_node_features, \
                num_layers=8, attn_type='performer', \
                    attn_kwargs=attn_kwargs).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00000001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, min_lr=0.0001)

# train test acc
for epoch in range(EPOCHS):
    loss = train_gps_graph(model, enzyme_train_loader, optimizer, device)
    test_acc = test_gps_graph(model, enzyme_train_loader, device)
    train_acc = test_gps_graph(model, enzyme_test_loader, device)
    scheduler.step(loss)

print(f'Final Loss: {loss:.4f},  Final Train Acc: {train_acc:.4f}, Final Test Acc: {test_acc:.4f}')


Final Loss: -0.1845,  Final Train Acc: 0.1750, Final Test Acc: 0.1333


# CORA

In [ ]:
# Load data
cora_dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]
num_node_features = cora_dataset.num_node_features
num_classes = int(cora_dataset.y.max().item()) + 1  # Assuming labels are from 0 to num_classes - 1

# init vars and params
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPSNode(
    num_node_features=num_node_features,
    hidden_channels=132,
    num_classes=num_classes,
    num_layers=8,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, min_lr=1e-4)

# training loop, get train test acc
for epoch in range(EPOCHS):
    loss = train_gps_nodes(model, cora_dataset, optimizer, device)
    train_acc, test_acc = test_gps_nodes(model, cora_dataset, device)
    scheduler.step(loss)

print(f'Final Loss: {loss:.4f}, Final Train Acc: {train_acc:.4f} , Final Test Acc: {test_acc:.4f}')


Final Loss: 0.0029, Final Train Acc: 1.0000 , Final Test Acc: 0.7260
